# Extracting Proper Noun

so far we used the pre-trained chunker to get the named-entity ,now we are going to design our own named entity.

In [1]:
import nltk
from nltk.corpus import treebank_chunk
from nltk.chunk import RegexpParser

In [2]:
def sub_leaves(tree,label):
    return [t.leaves() for t in tree.subtrees(lambda s:s.label() == label)]

a simple way to get the proper noun ,just by extracting the NNP

In [3]:
chunker =RegexpParser(
    """
    NAME : {<NNP>+} 
    """
)

In [4]:
tree = chunker.parse(treebank_chunk.tagged_sents()[0])

In [5]:
sub_leaves(tree,"NAME")

[[('Pierre', 'NNP'), ('Vinken', 'NNP')], [('Nov.', 'NNP')]]

Nov is name of the month. suppose we want only the name of person. we can use the use the "names corpus"

In [7]:
from nltk.corpus import names
from nltk.chunk import ChunkParserI
from nltk.chunk.util import conlltags2tree

In [8]:
class PersonChunker(ChunkParserI):
    def __init__(self) -> None:
        super().__init__()
        self.names = set(names.words())
        
    def parse(self, tokens):
        iobs = []
        in_person = False
        for word,tag in tokens:            
            is_name = word in self.names
            if is_name and in_person:
                iobs.append((word,tag,"I-PERSON"))
            elif is_name:
                iobs.append((word,tag,"B-PERSON"))
                in_person =True
            else:
                iobs.append((word,tag,"O"))
                in_person =False
        return conlltags2tree(iobs)

In [9]:
chunker = PersonChunker()
tree = chunker.parse(treebank_chunk.tagged_sents()[0])
sub_leaves(tree,"PERSON")

[[('Pierre', 'NNP')]]